In [ ]:
from evo.notebooks import ServiceManagerWidget

manager = await ServiceManagerWidget.with_auth_code(
    client_id="client-id",
).login()

In [ ]:
import uuid

import pandas as pd

from evo.objects.helpers.typed.pointset import PointSet

point_set = await PointSet.create(
    environment=manager.get_environment(),
    connector=manager.get_connector(),
    cache=manager.cache,
    name=f"My Point Set - {uuid.uuid4()}",
    data=pd.DataFrame(
        {
            "x": [1.0, 2.0, 3.0],
            "y": [4.0, 5.0, 6.0],
            "z": [7.0, 8.0, 9.0],
            "a": [10.0, 11.0, 12.0],
            "b": [False, True, False],
            "c": ["point1", "point2", "point3"],
        }
    ),
    coordinate_reference_system="EPSG:4326",
)

df = await point_set.as_dataframe()
df

In [ ]:
point_set.description = "This is my point set."
df.loc[0, "a"] = 99.9
df["d"] = [100, 200, 300]
await point_set.update_attributes(df)
await point_set.update()

In [ ]:
import json

from evo_schemas.elements.serialiser import GSONEncoder

print(point_set.metadata)

print(json.dumps(point_set.as_dict(), indent=2, cls=GSONEncoder))

df = await point_set.as_dataframe()
df